# Instrucciones.

Debería poder realizar todo el laboratorio en 3 horas.

Recomendamos fuertemente que siga trabajando en el laboratorio durante las horas asignadas, pues así puede consultar en vivo dudas que tenga sobre las partes finales del mismo.

Parte presencial.
* Complete la celda "A. Datos grupo".
* Descargue los archivos de instancias de ucursos.
* Complete las secciones B y C.
* Complete los ejercicios 1, 2 y 3 por el modulo de tareas de ucursos en la TAREA: Laboratorio 5 presencial
* Plazo de entrega: Miércoles 15 de julio a las 18:00

Parte no presencial.
* Complete todos los ejercicios (puede actualizar su parte presencial, pero debe INDICAR los cambios, agregándolos al final de la celda de Datos Grupo)
* Envie el archivo .ipynb por el modulo de tareas de ucursos en la TAREA: Laboratorio 5 completo
* Plazo de entrega: Domingo 19 de julio a las 23:59.

# A. Datos grupo

Escriba en esta celda el nombre de cada integrante de su grupo.

- Integrante 1: XXX

- Integrante 2: YYY

 # B. Preparación y Definición del problema


Usaremos los paquetes `JuMP, Gurobi, CSV, DataFrames`
Si le falta algún paquete instálelo (`using Pkg`; `Pkg.add("nombre_paquete")`) o actualícelo (`Pkg.update("nombre_paquete")`)
 
Al igual que en el laboratorio anterior, cree una constante `GUROBI_ENV` que guarde el ambiente gurobi que usaremos (de modo de evitar múltiples instancias). 

 

In [ ]:
# using Pkg
# Pkg.add(" ")
# Pkg.update(" ")

using JuMP, Gurobi, CSV, DataFrames
const GUROBI_ENV = Gurobi.Env()

Considere el siguiente problema.
Se recibe un conjunto de $n$ trabajos $J$. Queremos procesar los trabajos de $J$ en algún orden, en una máquina de modo que cierta función se minimice. Para esto necesitamos un poco de notación

* Cada trabajo $s$ en $J$ (piense que $s$ es un nombre), tiene asignado un **tiempo de proceso** $p_s \in \mathbb{N}^+$, y un **deadline** $d_s$.

* Una **agenda** es un ordenamiento de los trabajos $\pi\colon J \to [n]$, donde $\pi(s)=i$ si el $i$-ésimo trabajo del órden es el trabajo $s$. Decimos que un trabajo $s$ se procesa antes que un trabajo $t$ si $\pi(s)<\pi(t)$.

* El **tiempo de partida** $S^{\pi}_s$ de $s$ de acuerdo a una agenda $\pi$ es la suma de los tiempos de proceso de todos los trabajos que se procesan antes que $j$. 
$$S^{\pi}_s=\sum_{t\in J\colon \pi(t)<\pi(s)}p_t$$

* El **tiempo de completación** $C^{\pi}_s$ de $s$ de acuerdo a una agenda $\pi$ es su tiempo de partida más su tiempo de proceso: 
$$C^{\pi}_s=S^{\pi}_s+p_s =\sum_{t\in J\colon \pi(t)\leq \pi(s)}p_t$$

* La **tardanza** de un trabajo $s$ de acuerdo a $\pi$ es $T^{\pi}_s= \max(C^{\pi}_s-d_s, 0)$

El objetivo del problema de las tardanzas es encontrar la agenda que minimiza la tardanza total de todos los trabajos, es decir queremos resolver:
$$\min_{\pi} \sum_{s\in J} T^{\pi}_s$$

Hoy compararemos distintas formulaciones para el problema de las tardanzas (que es NP-difícil) e intentaremos mejorar algunas de ellas.


# C. Formulaciones #

## C1. Formulación indexada por tiempo.###

Como la duración de los trabajos es entera positivo, podemos discretizar el tiempo. Sabemos que los tiempos de completación deben ser enteros entre 1 y $𝑃=\sum_{s\in J} p_s$, e introducir variables binarias  $C_{s,i}$, que valen 1 si y solo si el trabajo $s$  termina en el tiempo  $i$.

Con estas variables el problema se puede formular como

\begin{align*}
\min \sum_{s\in J}\sum_{i=d_s+1}^{P} (i-d_s)C_{s,i}\\
\sum_{i=p_s}^{P}C_{s,i} &= 1, \forall s \in J\\
\sum_{s\in J}\sum_{j=i}^{\min(P,i+p_s-1)}C_{s,j} &\leq 1, \forall i \in [P]\\
C_{s,i}&\in \{0,1\}
\end{align*}



## C2. Mediante variables de tiempo de completación. ##

Usaremos variables $C_s, T_s$  que representen el tiempo de completación y la tardanza del trabajo $s$ (en rigor, como los tiempos de proceso son números naturales, podríamos fijar $C_s$ a ser entero, esto no es necesario). Además, introducimos variables binarias $y_{s,t}$ para todo $s, t \in J, s\neq t$ donde $y_{s,t}=1$ si $s$ se procesa antes que $t$.

El problema se puede formular como

\begin{align*}
\min &\sum_{s\in J} T_s\\
T_s&\geq C_s - d_s, \forall s \in J \\
T_s&\geq 0, \forall s \in J\\
C_s&\geq p_s, \forall s \in J\\
C_s+p_t&\leq C_t + Py_{t,s}, \forall s,t \in J\\
y_{s,t}+y_{t,s}&=1, \forall s,t \in J, s\neq t\\
y_{s,t}&\in \{0,1\}, \forall s,t \in J, s\neq t\\
\end{align*}

Donde $P=\sum_{s\in J}p_s$. 

## C3. Formulación por ordenes totales. ##

La idea de esta formulación es parecida a la anterior pero nos permite eliminar el coeficiente $P$ que normalmente dificulta la ejecución de un PLE (formulaciones tipo "big-M"). Usamos variables $y_{s,t}$ binarias que valgan 1 cuando el trabajo $s$
aparece antes que el trabajo $t$ (o $s=t$). Notamos que las variables definen un orden total si y solo si
hay reflexividad, antisimetría y transitividad. Esto se puede imponer pidiendo que para $r, s, t$ se tenga:
$y_{s,s}=1$, $y_{s,t}+y_{t,s}=1$ si $s\neq t$; y que $y_{s,t}+y_{t,r}-1\leq y_{s,r}$
La última ecuación se puede hacer más simétrica notando que solo la necesitamos cuando $s,t,r$ son distintos. En ese caso, podemos reemplazar $y_{s,t}$ por antisimetría y tenemos $y_{s,t}+y_{t,r}+y_{r,s}\leq 2$

\begin{align*}
\min &\sum_{s\in J} T_s\\
T_s&\geq C_s - d_s \\
T_s&\geq 0\\
C_s&=\sum_{t\in J}y_{t,s}p_t\\
y_{s,s}&=1\\
y_{s,t}+y_{t,s}&=1, \text{si $s\neq t$}\\
y_{s,t}+y_{t,r}+y_{r,s}&\leq 2, \text{si $r, s, t$ distintos}\\
y_{s,t}&\in \{0,1\}
\end{align*}





## C4. Instancias ##

La siguiente función permite leer instancias del problema de la tardanza.
Modo de uso 
```julia
proceso, deadline = LeeArchivo(nombrearchivo)
```

In [2]:
function LeeArchivo(archivocsv)
    M= Matrix(CSV.read(archivocsv,  header=0))
    return M[1,:], M[2,:]
end

LeeArchivo (generic function with 1 method)

# D. Ejercicios #


### Ejercicio 1 ###

Implementar las 3 formulaciones en una función 

```Julia
function Tardanza(i, proceso, deadline)
return modelo
```
donde "i" es 1, 2, o 3
proceso, deadline son arreglos de tamaño $n$ que tienen en su $i$-ésima coordenada, el tiempo de proceso y el deadline del trabajo $i$.

**IMPORTANTE** 
(si no implementa las siguientes caracteristicas tendrá un descuento importante en la revisión)
* Agregue como "warm start" la solución donde los trabajos están ordenados de $1$ a $n$ (En todos los modelos hay una forma de hacer esto de manera rápida).
* Ponga como cota fija de tiempo de ejecución de solver: 30 segundos.

Puede probar sus metodos en la instancia **instancia0.csv** que se lee como se indica  en la parte C4. El valor objetivo de dicha instancia es 237.

In [3]:
function Tardanza(i, proceso, deadline)
    ..
return modelo

LoadError: syntax: incomplete: "function" at none:1 requires end

### Ejercicio 2 ###

Implemente una función

```julia 
function Resuelve(i, proceso, deadline)
return valor, orden, tiempo, gap
```

donde la entrada es igual que en la parte anterior. Esta funcion debe devolver

valor  = valor objetivo encontrado.
orden  = un arreglo con un orden de los trabajos en la solucion encontrada
        Ejemplo: [1 3 4 2] significa que la agenda pone a 1 en primer lugar, 3 en segundo, 4 en tercero y 2 en último.
        (debido al warm-start de la parte anterior, su programa debería siempre encontrar solución).
tiempo = tiempo de ejecución usado
gap    = gap de resolución en porcentaje (0% = optimo)


## Symmetry Breaking ##

Las instancias provistas tienen mucha simetría. 

* Si dos trabajos $j$ y $k$ tienen igual tiempo de proceso  podemos "intercambiarlos" sin  alterar la tardanza de ningún trabajo aparte de $j$ y $k$ (todos quedan con su mismo tiempo de completación). Solo cambia la tardanza de $j$ y $k$. De hecho, si llamamos $\pi$ a la agenda antigua y $\tau$ a la agenda intercambiada entonces los tiempos de completación de $j$ y de $k$ se intercambian. Luego, la diferencia entre la función objetivo nueva y la antigua es (usamos $(a)_+=\max(a,0)$ y suponemos que $C^\pi_j < C^\pi_k$ )

\begin{align*}
T^\tau_j + T^\tau_k - T^\pi_j - T^\pi_j &= (C^\tau_j-d_j)_+ + (C^\tau_k-d_k)_+ - (C^\pi_j-d_j)_+ - (C^\pi_k-d_k)_+\\
&= (C^\pi_k-d_j)_+ +(C^\pi_j-d_k)_+ - (C^\pi_j-d_j)_+ - (C^\pi_k-d_k)_+
\end{align*}

No es difícil notar con esto que si el trabajo con menor deadline entre $d_j$ y $d_k$ se pone antes entonces la tardanza total es menor o igual que si el trabajo con menor deadline se pone después. 

En otras palabras, si dos trabajos tienen el mismo tiempo de proceso, podemos imponer un orden relativo a priori: los trabajos con menor deadline deben ir antes que los trabajos con mayor deadline.

Tomar decisiones a priori entre variables intercambiables es una manera de romper simetría que podría ayudar a que el árbol de BnB se simplifique (el solver no tiene que realizar el mismo trabajo en dos ramas que son iguales):


### Ejercicio 3. ###

Elija la formulación que quiera de las 3 y aplique symmetry breaking a los items con igual tiempo de proceso. Más precisamente: cuando escriba el modelo, particione los trabajos en grupos de trabajos con igual tiempo de proceso.
En cada grupo decida un orden fijo e imponga ese orden como restricciones adicionales.

En particular, cree funciones

```Julia
function TardanzaSB(duracion, peso, deadline)
return modelo
```
```julia 
function ResuelveSB(duracion, peso, deadline)
return valor, orden, tiempo, gap
```
que  hagan lo mismo que antes pero con esta formulación.

### Ejercicio 4 ###

En este problema deberá comparar las 3 formulaciones simples + la formulación con SB en las 10 instancias provistas 
instancia0.csv a instancia9.csv cada una con límite de tiempo de 30 segundos (y opciones estándar) 
    Más precisamente cree una función

```Julia
function Benchmark()
return df
```

donde df es un `dataframe` cuyas filas están indexadas por instancia, y las columnas son
    
| tiempo(1) | gap(1) | valor(1) | orden(1) | tiempo(2) | gap(2) | valor(2) | orden(2) | tiempo(3) | gap(3) | valor(3) | orden(3) | tiempo(SB) | gap(SB) | valor(SV) | orden(SB)

con
tiempo(j) = tiempo usado en la resolución de la instancia, usando formulacion 1
gap(j)    = gap de resolución en porcentaje (0% = optimo, infinito si no encontró solución factible)
valor(j)  = valor objetivo encontrado (infinito si no encontró solución)
orden(j)  = mejor orden encontrado 
